In [4]:
!pip install -Uqq chromadb
!pip install -Uqq langchain
!pip install -Uqq langchain-community

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, Language
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders.parsers.txt import TextParser
from langchain_community.document_loaders.text import TextLoader
from langchain.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain.embeddings import OllamaEmbeddings
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import RetrievalQA,RetrievalQAWithSourcesChain
from langchain import hub
from langchain_core.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
import os
import json
import shutil

In [6]:
DATA_PATH="code/"
DB_PATH = "vectorstores/db/"
model_name = 'codellama:70b'  
memory = ConversationBufferMemory(output_key='answer')

In [7]:
for root,dirs,files in os.walk(DB_PATH):
    for f in files:
        os.remove(f'{root}/{f}')
    for d in dirs:
        shutil.rmtree(f'{root}/{d}')

In [9]:
def create_vector_db():
    loader = DirectoryLoader(DATA_PATH, glob="**/*.cs",show_progress=True)
    documents = loader.load()         
    print(f"Processed {len(documents)} cs files")
    text_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.CSHARP,chunk_size=128, chunk_overlap=0)
    texts=text_splitter.split_documents(documents)
    vectorstore = Chroma.from_documents(documents=texts, embedding=OllamaEmbeddings(model=model_name, show_progress=True),persist_directory=DB_PATH)      
    #vectorstore.persist()

In [10]:
def retrieval_qa_chain(llm,vectorstore):
 qa_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    return_source_documents=True,
    memory = memory
)
 return qa_chain

In [11]:
def load_llm():
 llm = Ollama(
 model=model_name,
 verbose=True,
 callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
 )
 return llm

In [12]:
def qa_bot(): 
 llm=load_llm() 
 vectorstore = Chroma(persist_directory=DB_PATH, embedding_function=OllamaEmbeddings(model=model_name, show_progress=True))
 qa = retrieval_qa_chain(llm,vectorstore)
 return qa 

In [13]:
create_vector_db()

0it [00:00, ?it/s]


Processed 0 cs files


OllamaEmbeddings: 0it [00:00, ?it/s]


ValueError: Expected IDs to be a non-empty list, got 0 IDs

In [9]:
llm = load_llm()
llm

Ollama(verbose=True, callbacks=<langchain_core.callbacks.manager.CallbackManager object at 0x7fed24cdfb80>, model='llama3.1:70b')

In [10]:
chain=qa_bot()

In [11]:
rating_question = 'Based on how Paul performed please provide a rating "Achieves Expectations", "Exceeds Expectations", "Rockstar", or "Needs Improvement" for the review summary that was just written.'

In [12]:
review_questions = ['How does paul deliver work in line with the expectations for his role?',
                    'To what extent does paul achieve his set goals and how do these achievements contribute to the team’s and organization’s objective?',
                    'How does paul live the ISMs, demonstrate Antes to Succeed behaviors, and adhere to the expectations in the company Guideposts?',
                    'How does this paul demonstrate innovation and take initiative toward the team’s success?',
                    'How does paul actively mentor others and provide actionable feedback to create growth and impact?']
                    

In [13]:
final_rating_question = 'Based on the collected feedback and suammaries, please provide and overall performance rating ("Achieves Expectations", "Exceeds Expectations", "Rockstar", or "Needs Improvement") for this review period.' 

In [14]:
prompt_template = PromptTemplate.from_template(
    'Paul is a software architect.  Giving preference to Team Lead feedback write a concise review and cite four or five examples to answer the question "{question}"'
)

In [15]:
chain.invoke("Please describe the role and responsibilities of a software architect, according to Rocket Mortgage.  Do not include any feedback or notes documents.")

OllamaEmbeddings: 100%|███████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.30s/it]


The Software Architect at Rocket Mortgage acts as a technical thought leader who influences functional and non-functional requirements, contributes technical work items to the product roadmap, and oversees discussions with technical stakeholders. They are also responsible for elevating engineering team performance and have demonstrated their ability to engineer at a higher caliber.

SOURCES: data/role/software_architect.txt

{'question': 'Please describe the role and responsibilities of a software architect, according to Rocket Mortgage.  Do not include any feedback or notes documents.',
 'history': '',
 'answer': 'The Software Architect at Rocket Mortgage acts as a technical thought leader who influences functional and non-functional requirements, contributes technical work items to the product roadmap, and oversees discussions with technical stakeholders. They are also responsible for elevating engineering team performance and have demonstrated their ability to engineer at a higher caliber.\n\n',
 'sources': 'data/role/software_architect.txt',
 'source_documents': [Document(metadata={'source': 'data/role/software_architect.txt'}, page_content='The Software Architect acts as a technical thought leader with a deep understanding of the necessary trade-offs in the architectural design and integrations within their product area. They influence FRs and NFRs, contribute technical work items to the product roadm

In [16]:
saved_memory = memory

In [17]:
for q in review_questions:
    p = prompt_template.format(question=q)
    print(f'\n\nThe prompt is {p}\n\n')
    result = chain.invoke(p)
    print(result.keys())
    result = chain.invoke(rating_question)
    print(result.keys())



The prompt is Paul is a software architect.  Giving preference to Team Lead feedback write a concise review and cite four or five examples to answer the question "How does paul deliver work in line with the expectations for his role?"




OllamaEmbeddings: 100%|███████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.25s/it]


Paul delivers work in line with the expectations for his role by:

* Proposing engineering meetings to discuss data strategy and train on new technologies, showcasing his dedication to personal and team growth (Lisa Peploe's feedback)
* Bringing innovative topics to Train Architecture Sync, such as machine learning, and creating process improvements through discussions on integration testing (Lisa Peploe's feedback)
* Acting as a technical thought leader with a deep understanding of necessary trade-offs in architectural design and integrations within his product area (Software Architect role description)
* Elevating engineering team performance and demonstrating the ability to engineer at a higher caliber (Software Architect role description)
* Providing actionable feedback and demonstrating strong alignment with ISMs, Antes to Succeed, and Company Guideposts, thereby significantly contributing to the team's success (Exceeds Expectations rating description)

Overall, Paul consistently 

OllamaEmbeddings: 100%|███████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.30s/it]


Rockstar 

Paul consistently exceeds expectations with his technical expertise, proactive approach to championing change, and dedication to personal and team growth. He embodies the Antes to Succeed, Care Create, Connect, Deliver, and Grow, and has made significant contributions to the team's success through his effective communication, relationship-building skills, and initiative in creating process improvements.dict_keys(['question', 'history', 'answer', 'sources', 'source_documents'])


The prompt is Paul is a software architect.  Giving preference to Team Lead feedback write a concise review and cite four or five examples to answer the question "To what extent does paul achieve his set goals and how do these achievements contribute to the team’s and organization’s objective?"




...

Rockstardict_keys(['question', 'history', 'answer', 'sources', 'source_documents'])


The prompt is Paul is a software architect.  Giving preference to Team Lead feedback write a concise review and cite four or five examples to answer the question "How does paul actively mentor others and provide actionable feedback to create growth and impact?"




OllamaEmbeddings: 100%|███████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.32s/it]


Paul actively mentors others and provides actionable feedback in several ways:

1. **Proactive approach to championing change**: Paul proposes engineering meetings for discussing data strategy and training on new technologies, showing his dedication to personal and team growth (Source: Lisa Peploe's peer review).
2. **Effective communication and relationship-building skills**: He connects with team members and stakeholders, contributing significantly to the train's effectiveness, and brings incredible topics to the Train Architecture Sync (Source: Lisa Peploe's peer review).
3. **Initiative in creating process improvements**: Paul has multiple discussions on integration testing, which have been invaluable to the team (Source: Lisa Peploe's peer review).
4. **Technical foresight and suggestions for improvement**: He provides inspiring ideas and suggestions to improve solutions, making him a go-to person for challenging status-quo situations (Source: Gagan's peer review).

By demonstrati

OllamaEmbeddings: 100%|███████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.05s/it]


The rating for Paul based on the reviews provided by his peers (Gagan, Lisa Peploe, and Brandon Strong) would be "Rockstar".

Here's why:

* Gagan praises Paul as an inspiring personality who is always impressed with his technical foresight. He also appreciates how Paul creates opportunities for team members to share ideas and potential changes in the system.
* Lisa Peploe says that Paul is the definition of Antes to Succeed, Care Create, Connect, Deliver and Grow. She commends his proactive approach to championing change, his ability to connect with team members and stakeholders, and his effective communication and relationship-building skills.
* Brandon Strong describes Paul as a great human and someone he relies on for advice. He praises Paul's constant learning, vast knowledge of technologies and design patterns, and his growth mindset. He also appreciates how Paul asks great questions that drive discussions and offers valuable advice.

All these reviews suggest that Paul is an exc

In [18]:
result = chain.invoke(final_rating_question)
print(result.keys())

OllamaEmbeddings: 100%|███████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.13s/it]


Based on the collected feedback and summaries, I would rate Paul's overall performance as "Rockstar".

SOURCES:
- data/ratings/exceeds_expectations.txt
- data/feedback/lpeploe_20240618.txt
- data/feedback/gsharma_20240611.txtdict_keys(['question', 'history', 'answer', 'sources', 'source_documents'])


In [19]:
 print(memory)

chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Please describe the role and responsibilities of a software architect, according to Rocket Mortgage.  Do not include any feedback or notes documents.'), AIMessage(content='The Software Architect at Rocket Mortgage acts as a technical thought leader who influences functional and non-functional requirements, contributes technical work items to the product roadmap, and oversees discussions with technical stakeholders. They are also responsible for elevating engineering team performance and have demonstrated their ability to engineer at a higher caliber.\n\n'), HumanMessage(content='Paul is a software architect.  Giving preference to Team Lead feedback write a concise review and cite four or five examples to answer the question "How does paul deliver work in line with the expectations for his role?"'), AIMessage(content="Paul delivers work in line with the expectations for his role by:\n\n* Proposing engineering meeting

In [26]:
chain.invoke('Can you write a personal brand statements for Paul using all available information? The statement should be one to three sentences highlighting my skills, experience, and passion for my work. The five key elements should be my purpose, how do I set myself apart from others in this field, the promise that I make to my clients, how I use my personality, and how do I express my identity to others.')

OllamaEmbeddings: 100%|███████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.35s/it]


Here is a personal brand statement for Paul:

"As a driven and curious engineer, I excel at driving technical growth and innovation through strategic thinking, effective communication, and collaborative problem-solving. With a passion for learning and sharing knowledge, I empower teams to succeed by championing change, proposing process improvements, and delivering high-quality solutions that meet the needs of stakeholders. By combining technical expertise with strong interpersonal skills, I build trust and foster a culture of care, connection, and growth within my team and organization."

This statement incorporates the five key elements:

* Purpose: driving technical growth and innovation
* What sets you apart: strategic thinking, effective communication, and collaborative problem-solving
* Promise to clients: delivering high-quality solutions that meet their needs
* Personality: driven, curious, passionate about learning and sharing knowledge
* Identity: championing change, proposin

{'question': 'Can you write a personal brand statements for Paul using all available information? The statement should be one to three sentences highlighting my skills, experience, and passion for my work. The five key elements should be my purpose, how do I set myself apart from others in this field, the promise that I make to my clients, how I use my personality, and how do I express my identity to others.',
 'history': 'Human: Please describe the role and responsibilities of a software architect, according to Rocket Mortgage.  Do not include any feedback or notes documents.\nAI: The Software Architect at Rocket Mortgage acts as a technical thought leader who influences functional and non-functional requirements, contributes technical work items to the product roadmap, and oversees discussions with technical stakeholders. They are also responsible for elevating engineering team performance and have demonstrated their ability to engineer at a higher caliber.\n\n\nHuman: Paul is a soft

In [25]:
chain.invoke('using the statement "As a catalyst for growth and improvement, I ignite collaboration and creativity within teams. By sharing knowledge, resources, and expertise, I help organizations unlock their full potential and achieve success at scale." Can you expand this as a personal brand statement for paul?')

OllamaEmbeddings: 100%|███████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.42s/it]


Here is a potential personal brand statement for Paul:

"As a transformative technology leader, I empower teams and organizations to unlock their full potential through collaboration, creativity, and innovation. With a passion for growth and improvement, I ignite meaningful connections, drive strategic change, and deliver exceptional results at scale.

With expertise in software architecture, technical thought leadership, and team elevation, I bridge the gap between people, process, and technology to achieve remarkable outcomes. My dedication to personal and professional growth is contagious, inspiring others to strive for excellence and push beyond their limits.

Through my work, I aim to create a lasting impact on individuals, teams, and organizations, helping them navigate complex challenges, seize new opportunities, and thrive in an ever-changing world. Let's collaborate, innovate, and succeed together!"

This statement incorporates elements from the peer reviews and other sources,

{'question': 'using the statement "As a catalyst for growth and improvement, I ignite collaboration and creativity within teams. By sharing knowledge, resources, and expertise, I help organizations unlock their full potential and achieve success at scale." Can you expand this as a personal brand statement for paul?',
 'history': 'Human: Please describe the role and responsibilities of a software architect, according to Rocket Mortgage.  Do not include any feedback or notes documents.\nAI: The Software Architect at Rocket Mortgage acts as a technical thought leader who influences functional and non-functional requirements, contributes technical work items to the product roadmap, and oversees discussions with technical stakeholders. They are also responsible for elevating engineering team performance and have demonstrated their ability to engineer at a higher caliber.\n\n\nHuman: Paul is a software architect.  Giving preference to Team Lead feedback write a concise review and cite four o